In [1]:
# 필요한 라이브러리를 설치합니다.
# PyTorch 기반 SOTA 이미지 모델 라이브러리 ; resnet34 불러오기 위함
!pip install timm

In [2]:
import os
import time

import timm # 모델
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image # 이미지 입출력
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx): # DataLoader가 배치 단위로 호출할 때 행
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name))) # PIL.Image.open()으로 읽고 numpy array로 변환 img는 (H,W,C)형태 배열
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target # (이미지 텐서, 라벨) 반환 -> DataLoader에서 (batch_size, C, H, W)형태로 묶임

In [4]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0 # 에폭 전체 손실 합계 저장
    preds_list = [] # 모델 예측값 저장
    targets_list = [] # 정답(라벨) 저장

    pbar = tqdm(loader) # 진행 상태 출력
    for image, targets in pbar: # DataLoader에서 image, targets(라벨)을 배치 단위로 불러옴
        image = image.to(device) # 이미지를 GPU로 이동
        targets = targets.to(device)

        model.zero_grad(set_to_none=True) # 이전 배치에서 계산된 gradient 초기화, 메모리 최적화

        preds = model(image) # 모델 forward pass 수행으로 예측값 출력
        loss = loss_fn(preds, targets) # 예측값과 정답을 비교해 손실 계산
        loss.backward() # 역전파 파라미터별 gradient 계산
        optimizer.step() # gradient를 이용해 모델 파라미터 업데이트

        train_loss += loss.item() # 현재 배치 손실 누적
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

In [5]:
@torch.no_grad()
def valid_one_epoch(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0.0
    preds_all, t_all = [], []

    for images, targets in loader:
        images = images.to(device)
        targets = targets.to(device)

        logits = model(images)
        loss = loss_fn(logits, targets)
        val_loss += loss.item()

        preds = logits.argmax(1)
        preds_all.extend(preds.detach().cpu().numpy())
        t_all.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(t_all, preds_all)
    val_f1  = f1_score(t_all, preds_all, average="macro")
    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}


# 하이퍼파라미터

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda 하면 GPU 사용

# data config
data_path = '/root/cv_data/' # 데이터셋 저장 경로 : 이미지 폴더와 csv 파일들 위치

# model config
model_name = 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 224 # 입력 이미지 크기인데 32는 너무 작으니 224 정도로 바꾸기 
LR = 1e-3 # 학습률 AdamW은 1e-3 또는 5-4 / SGD는 1e-2 또는 1e-3 많이 사용
EPOCHS = 20 # 20~50 정도 돌려보기
BATCH_SIZE = 64 # 한번에 학습할 이미지 개수 -> GPU 메모리 용량에 따라 조정 64나 128이면 더 안정적인 gradient 추정
num_workers = 0 # DataLoader의 병렬 데이터 로딩 worker 수 0이면 메인 프로세스에서만 로딩(느려짐) 서버는 4~8 권장(코어 수 따라)

# 데이터 로드

In [7]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/root/cv_data/train.csv")

# 클래스 비율 유지하며 train/val 분리
trn_df, val_df = train_test_split(
    df,
    test_size = 0.2, # 8:2 분리
    stratify = df['target'], # 클래스 비율 유지
    random_state=42
)

# 임시 csv 저장
trn_df.to_csv('/root/cv_data/train_split.csv', index=False)
val_df.to_csv('/root/cv_data/val_split.csv', index=False)

In [ ]:
# Dataset 정의
trn_dataset = ImageDataset(
    "/root/cv_data/train_split.csv",
    "/root/cv_data/train",
    transform=trn_transform
)
val_dataset = ImageDataset(
    "/root/cv_data/val_split.csv",
    "/root/cv_data/train",
    transform=tst_transform # val데이터와 tst데이터는 같은 방식으로 변환해야 하기 때문
)
tst_dataset = ImageDataset(
    "/root/cv_data/sample_submission.csv",
    "/root/cv_data/test",
    transform=tst_transform
)
print(len(trn_dataset), len(val_dataset), len(tst_dataset))

1256 314 3140


In [14]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True, # 매 에폭마다 데이터 섞어서 배치 구성
    num_workers=num_workers,
    pin_memory=True, # gpu 전송을 빠르게 하기 위해 메모리 고정
    drop_last=False # 데이터 수가 배치 크기로 안 나눠떨어질 때 마지막 배치 버릴지 여부
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # 테스트 용은 데이터 순서 유지해야 함
    num_workers=0,
    pin_memory=True
)

In [15]:
# load model
model = timm.create_model(
    model_name,        # ex) "resnet34", "efficientnet-b0"
    pretrained=True,   # ImageNet 사전학습 가중치 사용
    num_classes=17     # 대회 클래스 개수
).to(device)

# 손실 함수 (클래스 불균형 심하면 weight 옵션 추가 가능)
loss_fn = nn.CrossEntropyLoss()

# Optimizer (Adam → AdamW)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,             # 초기 learning rate (예: 1e-3)
    weight_decay=1e-4  # 정규화 효과, 과적합 방지
)

# Scheduler (CosineAnnealingLR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,      # 전체 epoch 수와 맞춤
    eta_min=1e-6       # 최소 learning rate (0까지 떨어지지 않도록)
)


In [16]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [17]:
best_f1 = -1.0  # 가장 높은 val macro-F1 저장용, -1.0으로 초기화(첫 에포크 성능은 무조건 갱신되도록)

for epoch in range(EPOCHS):
    # 1) Train
    tr = train_one_epoch(trn_loader, model, optimizer, loss_fn, device)

    # 2) Validation 한 에포크 끝나고 검증
    va = valid_one_epoch(val_loader, model, loss_fn, device)

    # 3) Scheduler step (CosineAnnealingLR이면 epoch마다 호출 - 그래야 학습률 변함)
    scheduler.step()

    # 4) 로그 출력
    curr_lr = optimizer.param_groups[0]["lr"] # 현재 optimizer가 쓰는 학습률 가져옴
    print(
        f"[{epoch+1:02d}/{EPOCHS}] " # 두 자리 정수로 출력
        f"train_loss={tr['train_loss']:.4f}  train_f1={tr['train_f1']:.4f}  "
        f"val_loss={va['val_loss']:.4f}  val_f1={va['val_f1']:.4f}  lr={curr_lr:.6f}"
    )

    # 5) 베스트 모델 체크포인트 (대회 지표: macro-F1)
    if va["val_f1"] > best_f1: 
        best_f1 = va["val_f1"]
        torch.save(model.state_dict(), "best_resnet34.pth")
        print(f">> best updated! val_f1={best_f1:.4f}")


Loss: 1.4862: 100%|██████████| 20/20 [00:06<00:00,  3.09it/s]


[01/20] train_loss=2.2560  train_f1=0.4083  val_loss=1.5057  val_f1=0.5387  lr=0.000994
>> best updated! val_f1=0.5387


Loss: 0.5943: 100%|██████████| 20/20 [00:06<00:00,  3.19it/s]


[02/20] train_loss=0.8613  train_f1=0.7313  val_loss=0.5887  val_f1=0.7440  lr=0.000976
>> best updated! val_f1=0.7440


Loss: 0.2775: 100%|██████████| 20/20 [00:06<00:00,  3.21it/s]


...

[18/20] train_loss=0.0015  train_f1=1.0000  val_loss=0.2658  val_f1=0.9201  lr=0.000025


Loss: 0.0026: 100%|██████████| 20/20 [00:06<00:00,  3.24it/s]


[19/20] train_loss=0.0016  train_f1=1.0000  val_loss=0.2623  val_f1=0.9239  lr=0.000007


Loss: 0.0026: 100%|██████████| 20/20 [00:06<00:00,  3.22it/s]


[20/20] train_loss=0.0018  train_f1=1.0000  val_loss=0.2610  val_f1=0.9239  lr=0.000001


# 추론

In [18]:
# 1) best 모델 가중치 불러오기
model.load_state_dict(torch.load("best_resnet34.pth", map_location=device))
model.to(device)
model.eval()  # 평가 모드 전환

# 2) 추론 실행
preds_list = []
with torch.no_grad():  # 추론 시 gradient 계산 안 함, 한번에 감싸고 루프 전 모델 로드(가장 좋은 가중치로 추론하기 위함)
    for image, _ in tqdm(tst_loader):
        image = image.to(device)

        preds = model(image)  # (batch_size, num_classes) 출력
        preds = preds.argmax(dim=1)  # 가장 확률 높은 클래스 선택

        preds_list.extend(preds.detach().cpu().numpy())  # numpy로 변환 후 리스트에 추가


100%|██████████| 50/50 [00:13<00:00,  3.79it/s]


In [19]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [20]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,11
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,0
4,00901f504008d884.jpg,2


In [21]:
sample_submission_df = pd.read_csv("/root/cv_data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [22]:
pred_df.to_csv("code1_pred.csv", index=False)